In [21]:
import cv2
import numpy as np
import os
import threading
from threading import Thread
from matplotlib import pyplot as plt
import mediapipe as mp
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import tensorflow as tf
import playsound
import gtts
from gtts import gTTS

In [22]:
import unidecode
 
def remove_accent(text):
    return unidecode.unidecode(text)

In [23]:
def texttospeech(text, lang1):
    language = gtts.lang.tts_langs()
    if lang1 in language.keys():
        output = gTTS(text, lang=lang1, slow=False)
        output.save("output.mp3")
        playsound.playsound("output.mp3", True)
        os.remove("output.mp3")

    else:
        text = "Please choose correct languages you want"
        output = gTTS(text, lang="en", slow=False)
        playsound.playsound("output1.mp3", True)
        os.remove("output1.mp3")

In [24]:
cv2.__version__

'4.6.0'

In [25]:
mp_hands = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [26]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [27]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_hands.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_hands.HAND_CONNECTIONS) # Draw right hand connections

In [28]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_hands.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_hands.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [29]:
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

In [30]:
actions = os.listdir(os.path.join(os.getcwd(), 'Data'))

In [31]:
actions

['bạn hiểu ngôn ngữ ký hiệu không',
 'bạn học lớp mấy',
 'bạn khỏe không',
 'bạn muộn giờ rồi',
 'bạn phải cảnh giác',
 'bạn tiến bộ đấy',
 'bạn trông cau có thế',
 'bạn tên là gì',
 'bạn đang làm gì',
 'bạn đi đâu thế',
 'bố mẹ tôi cũng là người khiếm thính',
 'chúc mừng',
 'chúng tôi giao tiếp với nhau bằng ngôn ngữ ký hiệu',
 'con yêu mẹ',
 'cái này bao nhiêu tiền',
 'cái này là cái gì',
 'công việc của bạn là gì',
 'cảm ơn',
 'cấp cứu',
 'hẹn gặp lại các bạn',
 'món này không ngon',
 'tôi bị chóng mặt',
 'tôi bị cướp',
 'tôi bị kẹt xe',
 'tôi bị lạc',
 'tôi bị phân biệt đối xử',
 'tôi bị đau họng',
 'tôi bị đau đầu',
 'tôi cảm thấy rất hồi hộp',
 'tôi cảm thấy rất vui',
 'tôi cần gặp bác sĩ',
 'tôi cần phiên dịch',
 'tôi cần thuốc',
 'tôi cần ăn sáng',
 'tôi cần đi vệ sinh',
 'tôi không hiểu',
 'tôi không quan tâm',
 'tôi không đem theo tiền',
 'tôi khỏi bệnh rồi',
 'tôi là học sinh',
 'tôi là người khiếm thính',
 'tôi là thợ thêu',
 'tôi làm việc ở cửa hàng',
 'tôi nhầm địa chỉ',


In [32]:
model = load_model('action.h5')

In [33]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 60, 64)            48896     
                                                                 
 lstm_7 (LSTM)               (None, 60, 128)           98816     
                                                                 
 lstm_8 (LSTM)               (None, 64)                49408     
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 60)                1980      
                                                                 
Total params: 205,340
Trainable params: 205,340
Non-tr

In [34]:
import cv2   
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0) 
# Set mediapipe model 
with mp_hands.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read() # đọc video từ camera
        if frame is None:
            continue
        #print(frame)
        #plt.imshow(frame)
        # Make detections
        image, results = mediapipe_detection(frame, holistic) # gọi biến image và result = giá trị RGB của ảnh
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results) # nhận diện cơ thể 
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results) # xuất ra array các keypoints cơ thể (mấy cái đốt í)
        #print(keypoints)
        #sequence.insert(0,keypoints)
        #check = True
        values = np.array(keypoints)
        #values = map(list, int(values))
        #print(keypoints.all() == 0)
        #if (all(values) != 0):
        sequence.append(keypoints)
 
        sequence = sequence[-60:] # Đưa các frame thu đc vào mảng sequence, ở đây t để là 60 frame cuối
        
        if len(sequence) == 60:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)]) # Nếu thu đủ số frame của câu, đưa các array đấy vào model rồi ước lượng 
                                           # ra kết quả chính xác nhất trong dữ liệu
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 1: 
                sentence = sentence[-1:]

            # Viz probabilities
            # image = prob_viz(res, actions, image, colors)
        
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        print(keypoints)
        print(len(sequence))
        if len(sentence) > 0:
            cv2.putText(image, remove_accent(sentence[0]), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            if len(sequence) == 60:
                texttospeech(sentence[0], 'vi')
                #thread = Thread(target=texttospeech(sentence[0], 'vi'))
                #thread.start()
                sequence = []
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
[ 7.39062190e-01  1.02052236e+00 -4.83103975e-07  7.01199353e-01
  9.74237502e-01  1.15154656e-02  6.97945118e-01  9.16137457e-01
  2.03665793e-02  7.05553710e-01  8.72506082e-01  1.99813973e-02
  7.11277127e-01  8.37570071e-01  2.02125907e-02  7.44516253e-01
  8.72548342e-01  4.39412035e-02  7.43757725e-01  8.02503467e-01
  2.98820175e-02  7.13196456e-01  8.12490284e-01  1.80283319e-02
  6.95362687e-01  8.44809890e-01  1.40784420e-02  7.58840919e-01
  8.69244218e-01  3.07969395e-02  7.52029717e-01  7.87956476e-01
  1.17672123e-02  7.14613318e-01  8.05377126e-01 -3.64636420e-03
  6.96278453e-01  8.47710431e-01 -6.08089054e-03  7.79312074e-01
  8.69119048e-01  1.42371468e-02  7.63578176e-01  7.92269528e-01
 -5.03909076e-03  7.21382678e-01  8.16175699e-01 -1.58277210e-02
  7.01156437e-01  8.58349383e-01 -1.44958748e-02  8.06525409e-01
  8.74667287e-01 -1.85648713e-03  7.85350800e-01  8.15555632e-01
 -1.38943922e-02  7.46824384e-01 

<class 'mediapipe.python.solution_base.SolutionOutputs'>
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
9
<class 'mediapipe.python.solution_base.SolutionOutputs'>
[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000

<class 'mediapipe.python.solution_base.SolutionOutputs'>
[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 

<class 'mediapipe.python.solution_base.SolutionOutputs'>
[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 

<class 'mediapipe.python.solution_base.SolutionOutputs'>
[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 

...

<class 'mediapipe.python.solution_base.SolutionOutputs'>
[ 6.88314319e-01  9.07008827e-01 -3.43525301e-07  6.10359550e-01
  9.24435735e-01 -4.12670430e-04  5.27337134e-01  9.11199152e-01
 -3.36148706e-03  4.66259867e-01  9.20769095e-01 -1.59156844e-02
  4.18574750e-01  9.29645121e-01 -2.58791074e-02  5.31937122e-01
  7.50738502e-01  4.60108332e-02  4.84914601e-01  6.97105050e-01
  2.63684019e-02  4.48713481e-01  7.17797041e-01  4.79979208e-03
  4.28449392e-01  7.50294149e-01 -8.03478900e-03  5.50011516e-01
  7.19941676e-01  2.81998757e-02  4.95756269e-01  6.49837613e-01
  7.23689562e-03  4.56208169e-01  6.83466792e-01 -1.08171133e-02
  4.38239753e-01  7.31440663e-01 -1.85900480e-02  5.69278121e-01
  6.96892738e-01  3.06831906e-03  5.08270025e-01  6.31519794e-01
 -1.88107807e-02  4.63577986e-01  6.74692929e-01 -2.94964686e-02
  4.43456858e-01  7.27746844e-01 -3.04579958e-02  5.86344182e-01
  6.84689641e-01 -2.39243004e-02  5.22465706e-01  6.38172507e-01
 -3.67285237e-02  4.74292308e-01 

<class 'mediapipe.python.solution_base.SolutionOutputs'>
[ 6.95419967e-01  9.08832073e-01 -3.07350263e-07  6.13828242e-01
  9.26589072e-01  4.64734389e-03  5.31059444e-01  9.10827160e-01
  5.12871658e-03  4.71069425e-01  9.24783885e-01 -5.13790082e-03
  4.22813177e-01  9.35815334e-01 -1.26312356e-02  5.45281827e-01
  7.40676939e-01  5.98629676e-02  4.97612059e-01  6.84027433e-01
  4.81975079e-02  4.57479477e-01  7.02918410e-01  3.14390846e-02
  4.34029877e-01  7.36396730e-01  2.11269185e-02  5.61766684e-01
  7.13222742e-01  4.02136631e-02  5.04370749e-01  6.42203391e-01
  2.80878227e-02  4.59500074e-01  6.75406218e-01  1.30298836e-02
  4.37058091e-01  7.22679257e-01  6.15021819e-03  5.77659190e-01
  6.93736434e-01  1.29797701e-02  5.13019621e-01  6.27081394e-01
 -2.71042250e-03  4.62223798e-01  6.67804718e-01 -1.37256859e-02
  4.36636031e-01  7.19347239e-01 -1.63038727e-02  5.87558329e-01
  6.85297549e-01 -1.60109345e-02  5.20607889e-01  6.41686976e-01
 -2.55781710e-02  4.70552593e-01 

<class 'mediapipe.python.solution_base.SolutionOutputs'>
[ 6.98231816e-01  9.11507726e-01 -2.97985167e-07  6.17884159e-01
  9.28203464e-01  1.08659882e-02  5.38134694e-01  9.09691751e-01
  1.41392127e-02  4.77560103e-01  9.25769448e-01  5.01576532e-03
  4.29810077e-01  9.46709573e-01 -1.97298243e-03  5.53901017e-01
  7.37725854e-01  7.00150952e-02  5.06114185e-01  6.78838730e-01
  6.15301728e-02  4.63937163e-01  6.93098545e-01  4.59450260e-02
  4.38638866e-01  7.23694086e-01  3.58959697e-02  5.67671776e-01
  7.11674273e-01  4.71057221e-02  5.09219825e-01  6.38624251e-01
  3.78971957e-02  4.62612897e-01  6.68257654e-01  2.31395867e-02
  4.40181404e-01  7.15823054e-01  1.59263369e-02  5.81100225e-01
  6.94186151e-01  1.67496148e-02  5.14361739e-01  6.27854764e-01
  2.97858543e-03  4.64006752e-01  6.67366385e-01 -7.90502038e-03
  4.39945906e-01  7.18333125e-01 -1.07822828e-02  5.87926686e-01
  6.87719524e-01 -1.49547709e-02  5.18982291e-01  6.43362880e-01
 -2.23815273e-02  4.70746487e-01 

<class 'mediapipe.python.solution_base.SolutionOutputs'>
[ 6.96817756e-01  9.09618497e-01 -3.16645895e-07  6.22106493e-01
  9.28101659e-01  8.14567227e-03  5.43575883e-01  9.11071539e-01
  1.18546942e-02  4.86618310e-01  9.32138264e-01  4.05271305e-03
  4.42215890e-01  9.61241364e-01 -1.98233873e-03  5.53336143e-01
  7.45129526e-01  6.65545240e-02  5.05485773e-01  6.92321599e-01
  5.81332669e-02  4.65589315e-01  7.09833384e-01  4.28565256e-02
  4.40894485e-01  7.39974618e-01  3.26500237e-02  5.66163003e-01
  7.18516290e-01  4.54735942e-02  5.08716762e-01  6.48691058e-01
  3.52839865e-02  4.64674383e-01  6.79683208e-01  1.98337212e-02
  4.41587210e-01  7.26143837e-01  1.23291723e-02  5.80042481e-01
  6.98707044e-01  1.70975849e-02  5.13640463e-01  6.34583056e-01
  1.32198585e-03  4.65105563e-01  6.76588595e-01 -1.10021243e-02
  4.39611256e-01  7.27608263e-01 -1.42181152e-02  5.88524103e-01
  6.88598990e-01 -1.25834914e-02  5.18976927e-01  6.48782372e-01
 -2.20567342e-02  4.71560985e-01 

1/1 [==============================] - 0s 25ms/step
tôi bị phân biệt đối xử
[ 6.90349579e-01  9.05857742e-01 -2.97031164e-07  6.11579895e-01
  9.26061749e-01 -7.00262201e-04  5.24927258e-01  9.13266301e-01
 -8.54838348e-04  4.58047569e-01  9.25251126e-01 -1.01409582e-02
  4.05423760e-01  9.32997763e-01 -1.64627228e-02  5.38551390e-01
  7.52920687e-01  5.40419146e-02  4.92729723e-01  6.98768616e-01
  4.02105525e-02  4.52851027e-01  7.15548158e-01  2.20668595e-02
  4.29834187e-01  7.46541500e-01  1.07044317e-02  5.55336177e-01
  7.22986698e-01  3.79524827e-02  5.02442658e-01  6.50721133e-01
  2.22105905e-02  4.60406989e-01  6.78526640e-01  6.05170010e-03
  4.39676046e-01  7.23791182e-01 -1.30256498e-03  5.72979152e-01
  7.00941086e-01  1.39745222e-02  5.10456085e-01  6.36827528e-01
 -6.42132573e-03  4.63593036e-01  6.74385726e-01 -1.83879286e-02
  4.40592825e-01  7.24776864e-01 -2.04899870e-02  5.85705876e-01
  6.89354002e-01 -1.17255244e-02  5.18486083e-01  6.48927629e-01
 -2.56668236e-